In [0]:
/* Databricks SQL Query: ADS_RDS_EVENT_TYPES_REGISTRATIONACTION */
/* Converted from notebook format for better performance */
/* Generated: 2025-09-23 09:33:17 */

/* Declare variables */ 
DECLARE OR REPLACE VARIABLE p_load_date STRING DEFAULT {{p_load_date}};
DECLARE OR REPLACE VARIABLE p_process_key STRING DEFAULT {{p_process_key}};
DECLARE OR REPLACE VARIABLE map_id STRING DEFAULT 'ADS_RDS_EVENT_TYPES_REGISTRATIONACTION';
DECLARE OR REPLACE VARIABLE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_REGISTRATIONACTION_ADS_MAP_SCD_DIFF';
DECLARE OR REPLACE VARIABLE max_key BIGINT DEFAULT 0;

-- Get the maximum EVETP_KEY from Target table
/*
SET VAR max_key = (
    SELECT COALESCE(MAX(evetp_key), 0) 
    FROM gap_catalog.ads_owner.event_types
);
*/

/* Truncate XC Table */
TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_REGISTRATIONACTION;

/* Fill XC Table */
INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_REGISTRATIONACTION 
      with C_0FILTERAJDNR1FIIIND9JFP621UP020CF as (
      select 
          RDS_IDENTITYREGISTRATIONACTION.value  VALUE
        from gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_IDENTITYREGISTRATIONACTION RDS_IDENTITYREGISTRATIONACTION
        where ( RDS_IDENTITYREGISTRATIONACTION.sys   = 'Brasil'
            and RDS_IDENTITYREGISTRATIONACTION.lang  = 'EN'
            and CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
              )
      )
      select  
              /*seznam vkladanych nebo updatovanych sloupcu bez SK, auditnich atribudu a deleted flagu*/
          FILTER_A.VALUE as EVETP_SOURCE_ID,
          'RDS' as EVETP_SOURCE_SYSTEM_ID,
          'RDS_IDENTITYREGISTRATIONACTION' as EVETP_SOURCE_SYS_ORIGIN,
          FILTER_A.VALUE as EVETP_DESC,
          'PROCESS_EVENTS' as EVETP_EVENT_TABLE_NAME,
          'XNA' as EVETP_TARGET,
          'XNA' as EVETP_DATA_PATH,
          'N' as EVETP_DELETED_FLAG
        from (C_0FILTERAJDNR1FIIIND9JFP621UP020CF FILTER_A)
        where (1=1)
;

/* Cleanup DIFF Table */
DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_REGISTRATIONACTION_ADS_MAP_SCD_DIFF;

/* Create DIFF Table */
create  table gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_REGISTRATIONACTION_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  EVETP_KEY  INTEGER,
  EVETP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  EVETP_SOURCE_ID  varchar(400),
  EVETP_SOURCE_SYSTEM_ID  varchar(120),
  EVETP_SOURCE_SYS_ORIGIN  varchar(120),
  EVETP_DESC  varchar(4000),
  EVETP_EVENT_TABLE_NAME  varchar(120),
  EVETP_TARGET  varchar(120),
  EVETP_DATA_PATH  varchar(120))
;

/* Populate DIFF Table - New/Updated Records */
insert into IDENTIFIER(dif_table_name)
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  EVETP_KEY,
  EVETP_SOURCE_ID,
  EVETP_SOURCE_SYSTEM_ID,
  EVETP_SOURCE_SYS_ORIGIN,
  EVETP_DESC,
  EVETP_EVENT_TABLE_NAME,
  EVETP_TARGET,
  EVETP_DATA_PATH
)
select   'N' as tech_del_flg, 
    case when trg.EVETP_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
    trg.rid as tech_rid,
   trg.EVETP_KEY, 
   src.EVETP_SOURCE_ID, 
   src.EVETP_SOURCE_SYSTEM_ID, 
   src.EVETP_SOURCE_SYS_ORIGIN, 
   src.EVETP_DESC, 
   src.EVETP_EVENT_TABLE_NAME, 
   src.EVETP_TARGET, 
   src.EVETP_DATA_PATH
 from 
    (select EVETP_SOURCE_ID, 
       EVETP_SOURCE_SYSTEM_ID, 
       EVETP_SOURCE_SYS_ORIGIN, 
       EVETP_DESC, 
       EVETP_EVENT_TABLE_NAME, 
       EVETP_DELETED_FLAG, 
       EVETP_TARGET, 
       EVETP_DATA_PATH
       from gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_REGISTRATIONACTION xc
    where (EVETP_SOURCE_SYS_ORIGIN='RDS_IDENTITYREGISTRATIONACTION')) src LEFT JOIN
    (select  evetp_key||'.'||evetp_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_TYPES t
      where EVETP_CURRENT_FLAG  = 'Y'
        and EVETP_VALID_TO  = to_date('01013000','ddMMyyyy')
       and (EVETP_SOURCE_SYS_ORIGIN='RDS_IDENTITYREGISTRATIONACTION')      ) trg
ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
 and trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
 and trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
 and trg.EVETP_VALID_TO = to_date('30000101','yyyyMMdd') WHERE (
     decode( src.EVETP_DESC,trg.EVETP_DESC,1,0 ) = 0  or
     decode( src.EVETP_EVENT_TABLE_NAME,trg.EVETP_EVENT_TABLE_NAME,1,0 ) = 0  or
     decode( src.EVETP_DELETED_FLAG,trg.EVETP_DELETED_FLAG,1,0 ) = 0  or
     decode( src.EVETP_TARGET,trg.EVETP_TARGET,1,0 ) = 0  or
     decode( src.EVETP_DATA_PATH,trg.EVETP_DATA_PATH,1,0 ) = 0 or 
     trg.EVETP_SOURCE_ID is null   );

/* Populate DIFF Table - Deleted Records */
insert into IDENTIFIER(dif_table_name)
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  EVETP_KEY,
  EVETP_SOURCE_ID,
  EVETP_SOURCE_SYSTEM_ID,
  EVETP_SOURCE_SYS_ORIGIN,
  EVETP_DESC,
  EVETP_EVENT_TABLE_NAME,
  EVETP_TARGET,
  EVETP_DATA_PATH
)
select 'Y' as tech_del_flg, 
 'N' as tech_new_rec, 
 trg.rid as tech_rid, 
   trg.EVETP_KEY, 
   trg.EVETP_SOURCE_ID, 
   trg.EVETP_SOURCE_SYSTEM_ID, 
   trg.EVETP_SOURCE_SYS_ORIGIN, 
   trg.EVETP_DESC, 
   trg.EVETP_EVENT_TABLE_NAME, 
   trg.EVETP_TARGET, 
   trg.EVETP_DATA_PATH
 from 
    (select  evetp_key||'.'||evetp_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_TYPES t
      where EVETP_CURRENT_FLAG  = 'Y' and EVETP_DELETED_FLAG  = 'N'
        and EVETP_VALID_TO  = to_date('01013000','ddMMyyyy')
       and (EVETP_SOURCE_SYS_ORIGIN='RDS_IDENTITYREGISTRATIONACTION')      ) trg LEFT JOIN
    (select   EVETP_SOURCE_ID, 
       EVETP_SOURCE_SYSTEM_ID, 
       EVETP_SOURCE_SYS_ORIGIN, 
       to_date('30000101','yyyyMMdd') as EVETP_VALID_TO
       from gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_REGISTRATIONACTION xc
    where (EVETP_SOURCE_SYS_ORIGIN='RDS_IDENTITYREGISTRATIONACTION')) src
ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
 and trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
 and trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
 and trg.EVETP_VALID_TO = src.EVETP_VALID_TO WHERE (src.EVETP_SOURCE_ID is null);

/* Close Old Records in Target */
update  gap_catalog.ads_owner.EVENT_TYPES set
       EVETP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       EVETP_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
       EVETP_CURRENT_FLAG = 'N', 
       EVETP_VALID_TO = to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd')-1
 where EVETP_CURRENT_FLAG = 'Y'
   and EVETP_VALID_TO = to_date('30000101','yyyyMMdd')
   and evetp_key||'.'||evetp_valid_from in (select tech_rid from IDENTIFIER(dif_table_name) where tech_rid is not null);

/* Insert Changed Records */
insert  into gap_catalog.ads_owner.EVENT_TYPES 
 ( EVETP_KEY, 
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   EVETP_VALID_FROM, 
   EVETP_VALID_TO, 
   EVETP_CURRENT_FLAG, 
   EVETP_DELETED_FLAG, 
   EVETP_INSERTED_DATETIME, 
   EVETP_INSERT_PROCESS_KEY, 
   EVETP_UPDATED_DATETIME, 
   EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH)
select  EVETP_KEY, 
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd') as EVETP_VALID_FROM, 
   DATE('3000-01-01') as EVETP_VALID_TO, 
   'Y' as EVETP_CURRENT_FLAG, 
   tech_del_flg as EVETP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH
  from IDENTIFIER(dif_table_name)
 where tech_new_rec = 'N';

/* Insert New Records */
insert  into gap_catalog.ads_owner.EVENT_TYPES 
 ( EVETP_KEY, 
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   EVETP_VALID_FROM, 
   EVETP_VALID_TO, 
   EVETP_CURRENT_FLAG, 
   EVETP_DELETED_FLAG, 
   EVETP_INSERTED_DATETIME, 
   EVETP_INSERT_PROCESS_KEY, 
   EVETP_UPDATED_DATETIME, 
   EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH)
select 
  -- EVETP_KEY_NEW + max_key AS EVETP_KEY,  -- Add the offset to maintain key sequence, 
   xxhash64(uuid()) AS EVETP_KEY,
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   to_date(CAST(p_load_date AS DATE),'yyyy-MM-dd') as EVETP_VALID_FROM, 
   DATE('3000-01-01') as EVETP_VALID_TO, 
   'Y' as EVETP_CURRENT_FLAG, 
   tech_del_flg as EVETP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
   CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH
  from IDENTIFIER(dif_table_name)
 where tech_new_rec = 'Y';